# Capstone Project

This notebook will be used for the Capstone project of IBM's Data Science Specialization on Coursera

In [1]:
import numpy as np
import pandas as pd

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto
### Loading and cleaning data

In [3]:
# extract tables from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = pd.read_html(url,  attrs={"class":"wikitable"})
print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [4]:
# Renaming the headers
df = wikitables[0]
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [5]:
# Remove the 'Not assigned' Boroughs
df = df[ df['Borough'] != 'Not assigned']

# Replace the 'Not assigned' Neighbourhoods with their corresponding Boroughs names
for i, item in enumerate(df['Neighbourhood']):
    if item == 'Not assigned':
        df['Neighbourhood'].iloc[i] = df['Borough'].iloc[i]
        
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [6]:
# Now lets merge Neighbourhoods with similar postcode numbers
merged_neighbourhoods_df = df.groupby('Postcode').Neighbourhood.apply(lambda x: "%s" % ', '.join(x)).to_frame().reset_index()
df['Neighbourhood'] = merged_neighbourhoods_df['Neighbourhood']
df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Woburn
1,M4A,North York,Cedarbrae
2,M5A,Downtown Toronto,Scarborough Village
3,M6A,North York,"East Birchmount Park, Ionview, Kennedy Park"
4,M6A,North York,"Clairlea, Golden Mile, Oakridge"


In [7]:
geo_coord = pd.read_csv('Geospatial_Coordinates.csv')
geo_coord.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
geo_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df = pd.merge(df, geo_coord, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Woburn,43.753259,-79.329656
1,M4A,North York,Cedarbrae,43.725882,-79.315572
2,M5A,Downtown Toronto,Scarborough Village,43.654260,-79.360636
3,M6A,North York,"East Birchmount Park, Ionview, Kennedy Park",43.718518,-79.464763
4,M6A,North York,"Clairlea, Golden Mile, Oakridge",43.718518,-79.464763


### preprocessing data

In [9]:
import folium
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [11]:
address = 'Toronto'
geolocator = Nominatim()

try:
    location = geolocator.geocode(address, timeout=30)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(address, e.msg))

C:\Users\Toshiba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'W2K3FISCHVADQ5Q0EU4YYR50ASUUEHSVK0LLNSQ03P3YN320' # your Foursquare ID
CLIENT_SECRET = 'VZQYGDL05PJ5SSLQ0NDKHF0F4Y44C5AN0K2Y2CRBZ1K2U23O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W2K3FISCHVADQ5Q0EU4YYR50ASUUEHSVK0LLNSQ03P3YN320
CLIENT_SECRET:VZQYGDL05PJ5SSLQ0NDKHF0F4Y44C5AN0K2Y2CRBZ1K2U23O


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
#### First, let's create the GET request URL. Name your URL url

In [14]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude,
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=W2K3FISCHVADQ5Q0EU4YYR50ASUUEHSVK0LLNSQ03P3YN320&client_secret=VZQYGDL05PJ5SSLQ0NDKHF0F4Y44C5AN0K2Y2CRBZ1K2U23O&v=20180605&ll=43.653963,-79.387207&radius=500&limit=100'

In [ ]:
# Send the GET request and examine the resutls
results = requests.get(url).json()
results

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Cafe Plenty,Café,43.654571,-79.389450
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Rolltation,Japanese Restaurant,43.654918,-79.387424
4,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

77 venues were returned by Foursquare.


In [19]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 53 uniques categories.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    i = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        print(name)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        i += 1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

toronto_venues.head()

Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Harbourfront
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Bedford Park, Law

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Woburn,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Cedarbrae,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Cedarbrae,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Cedarbrae,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [26]:
toronto_venues.rename(columns={'Neighborhood': 'Neighbourhood'}, inplace=True)
print(toronto_venues.shape)
toronto_venues.head()

(2171, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Woburn,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Cedarbrae,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Cedarbrae,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Cedarbrae,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [27]:
toronto_venues.groupby('Neighbourhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",17,17,17,17,17,17
Agincourt,1,1,1,1,1,1
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",5,5,5,5,5,5
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",22,22,22,22,22,22
"Bathurst Manor, Downsview North, Wilson Heights",1,1,1,1,1,1


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


### Analyze Each Neighborhood

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_onehot.shape

(2171, 211)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
# Now let's create a new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Grocery Store,Park,Athletics & Sports,Italian Restaurant,Candy Store,Restaurant,Baby Store,Diner,Nightclub
1,Agincourt,Fast Food Restaurant,College Cafeteria,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Donut Shop,Dog Run,Discount Store,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Japanese Restaurant,Café,Convenience Store,Comic Shop,Event Space,Ethiopian Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Breakfast Spot,Café,Coffee Shop,Performing Arts Venue,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
4,"Bathurst Manor, Downsview North, Wilson Heights",Bar,Yoga Studio,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Donut Shop,Dog Run,Discount Store


Run k-means to cluster the neighborhood into 5 clusters

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 4, 2, 2, 2, 3, 2])

In [47]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df
# del toronto_merged['Borough']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(axis=0, inplace=True)

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Woburn,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Donut Shop,Dog Run,Discount Store,Diner,Dim Sum Restaurant
1,M4A,North York,Cedarbrae,43.725882,-79.315572,2.0,Grocery Store,Coffee Shop,Hockey Arena,Nail Salon,Portuguese Restaurant,Dance Studio,Electronics Store,Donut Shop,Dog Run,Discount Store
2,M5A,Downtown Toronto,Scarborough Village,43.654260,-79.360636,2.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Mexican Restaurant,Chocolate Shop,Dessert Shop
3,M6A,North York,"East Birchmount Park, Ionview, Kennedy Park",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Gift Shop,Arts & Crafts Store,Vietnamese Restaurant,Coffee Shop,Boutique,Ethiopian Restaurant
4,M6A,North York,"Clairlea, Golden Mile, Oakridge",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Gift Shop,Arts & Crafts Store,Vietnamese Restaurant,Coffee Shop,Boutique,Ethiopian Restaurant


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters